# early_whales

> give this a good description

In [ ]:
#|default_exp early_whales

In [ ]:
#| hide
from nbdev.showdoc import *

**Functionality of `early_whales`:**

Fill me in


Definition 1: A  (early?) `whale` is a portfolio value across a threshold ~ the date a token was bought. e.g. if an address is the 13th one to purchase `pepe` (so an early buyer) and on the date they first did, their total wallet value was $> x$ USD for some large enough $x$ then we call them an `early whale`.

In [ ]:
#| export

from api_endpoints.etherscan_api import *
from api_endpoints.covalent_api import *
from etherscan import Etherscan #externally installed package https://github.com/pcko1/etherscan-python

from typing import List, Dict
from fastcore.basics import *
from fastcore.test import *
import json


Some utility functions:

In [ ]:
#| export
from typing import Union

def is_whale(list_of_dict:List[dict],threshold:int)->List[dict]:
    """
        Determines whether each address is a whale or not.
        Args:
            list_of_dict: list of dictionaries, where each dict has keys 'address', 'portfolio' and 'portfolio_sum'. 
                          For example, `list_of_dict` could be the output of `get_holders_portfolios` function.
            threshold: threshold value to determine whether an address is a whale or not

        Returns: 
            list_of_dict: updates the input list_of_dict with a new key 'is_whale' which is True if the address is a whale, False otherwise (except
                          if the portfolio_sum is None, in which case is_whale is None).
    """

    for _holder in list_of_dict:
        _holder['is_whale'] =( _holder['portfolio_sum'] >= threshold if  _holder['portfolio_sum']!=None else None)

    return list_of_dict

def print_whales_util(input_obj: Union["EarlyWhales", dict]) -> None:
    """Prints the whales in a nice format."""
    
    # Check if input_obj is an instance of EarlyWhales
    if isinstance(input_obj, EarlyWhales):
        whale_list = input_obj.whale_list
        n = input_obj.n
        threshold = input_obj.threshold
    
    # Else, assume it's a dictionary
    else:
        whale_list = input_obj.get('whale_list')
        n = input_obj.get('n')
        threshold = input_obj.get('threshold')
        
    print(f"Using a threshold of {threshold} USD, the following addresses out of the first {n} are whales:")
    for _holder in whale_list:
        # Assuming you want to keep this test
        test_eq(_holder.get('is_whale'),True)
        print(f"Address: {_holder['address']} had a portfolio value of {_holder['portfolio_sum']}")


In [ ]:
#| export

class EarlyWhales:
    "Wrapper class. Basically a map from `address`,`n`,`threshold` to a list of whales."
    def __init__(self, address: str, n: int, threshold: int, cov_api: Covalent_Api, chainName: str, network: str,etherscan_api_key:str): 
        self._threshold = threshold  # Initialize this without invoking the property setter
        self.address = address
        self.n = n
        self.cov_api = cov_api
        self.chainName = chainName
        self.network = network
        self.etherscan_api_key = etherscan_api_key
        self.get_data()
    
    def get_data(self) -> None:
        """Initializes data by making API calls and calculations."""
        self.first_n = get_first_n_addresses(self.address, self.n, self.etherscan_api_key)
        
        self.first_n_addresses = [address for address in self.first_n]
       
        self.creation_date = get_creation_date(self.address, self.etherscan_api_key, network=self.network).split()[0]
        
        self._first_n_addresses_lstofdict = [{'address': address} for address in self.first_n_addresses]
        
        self._first_n_addresses_lstofdict = self.cov_api.get_holders_portfolios(
            wallet_list=self._first_n_addresses_lstofdict, chainName=self.chainName, quote_currency="USD", date=self.creation_date, log_output=False)
                        
        self.update_first_n_addresses_lstofdict()

        self.update_whale_list()

    def update_first_n_addresses_lstofdict(self) -> None:
        """Updates the list of dictionaries based on the current threshold."""

        self._first_n_addresses_lstofdict = is_whale(self._first_n_addresses_lstofdict, threshold=self.threshold)

    def update_whale_list(self) -> None:
        """Updates the list of whales based on the current threshold."""
        self._whale_list = [holder for holder in self._first_n_addresses_lstofdict if holder['is_whale']]
        
    @property
    def threshold(self) -> int:
        """Gets or sets the threshold for determining whales."""
        return self._threshold 

    @threshold.setter
    def threshold(self, val: int) -> None:
        """
        Sets the threshold for determining whales and updates the list of dictionaries
        to reflect the new threshold. A change in threshold will potentially yield
        a different collection of "whales."
        """
        #The point is, when you change the threshold, you also need to update the list of dictionaries
        self._threshold = val
        self.update_first_n_addresses_lstofdict() #the _holder['is_whale'] value will be updated
        self.update_whale_list() #the _whale_list will be updated

    @property
    def first_n_addresses_lstofdict(self) -> List[Dict]:
        """Read-only property that returns the list of first N addresses."""
        return self._first_n_addresses_lstofdict
    
    @property
    def whale_list(self) -> List[Dict]:
        """Read-only property that returns the list of whales."""
        return self._whale_list
    
    def print_whales(self) -> None:
        """Prints the whales in a nice format."""
        print_whales_util(self)



It's helpful to convert an `EarlyWhales` object to a dictionary so we can save and load the json file:

In [ ]:
#| export
from copy import deepcopy

#Once you have an EarlyWhales object, you can convert it to a dictionary. This is nice as we can easily e.g. 
#save the object to a json file, and explore it later.
def earlywhales_to_dict(earlywhales:EarlyWhales)->Dict:
    
    """Converts an EarlyWhales object to a dictionary. This is useful for saving the object to a json file, and
        exploring it later. We want this to be a pure function to avoid side effects.
    """
    
    return deepcopy({'address': earlywhales.address,
            'n': earlywhales.n,
            'threshold': earlywhales.threshold,
            'chainName': earlywhales.chainName,
            'network': earlywhales.network,
            'creation_date': earlywhales.creation_date,
            'first_n_addresses': earlywhales.first_n_addresses,
            'first_n_addresses_lstofdict': earlywhales.first_n_addresses_lstofdict,
            'whale_list': earlywhales.whale_list
            })

#Simple utility method if we want to change the threshold in an earlywhales_dict.
#We need to of course update the whales if we do so.
def earlywhales_dict_change_threshold(earlywhales_dict:Dict, new_threshold:int)->Dict:
    """Changes the threshold of an EarlyWhales object, and returns the updated dictionary.
       We want this to be a pure function to avoid side effects.
    """
    
    earlywhales_dict = deepcopy(earlywhales_dict)
    earlywhales_dict['threshold'] = new_threshold
    earlywhales_dict['first_n_addresses_lstofdict'] = is_whale(earlywhales_dict['first_n_addresses_lstofdict'], threshold=new_threshold)
    earlywhales_dict['whale_list'] = [holder for holder in earlywhales_dict['first_n_addresses_lstofdict'] if holder['is_whale']]
    return earlywhales_dict

def save_dict_to_json(data_dict: dict, file_path: str) -> None:
    """
    Saves a dictionary to a JSON file.

    Parameters:
        data_dict (dict): The dictionary to save.
        file_path (str): The path where the JSON file will be saved.
    """
    with open(file_path, 'w') as f:
        json.dump(data_dict, f)


def load_dict_from_json(file_path: str) -> dict:
    """
    Loads a dictionary from a JSON file.

    Parameters:
        file_path (str): The path where the JSON file is located.

    Returns:
        dict: The loaded dictionary.
    """
    with open(file_path, 'r') as f:
        data_dict = json.load(f)
    return data_dict



How to use `EarlyWhales` - please view the next several cells which has information on how to use this class (including the conversion to dict and saving utility methods).

In [ ]:
# ###setup
# n=500
# chainName='eth-mainnet'
# network='mainnet'
# cov_api = Covalent_Api() #may need to enter an API key into the Covalent_Api class if our default isn't working
# #etherscan_api = #may need to enter an API key
# threshold=100000
# ###

# early_pepe_whales = EarlyWhales(address=pepe_address, 
#                                 n=n,
#                                 threshold=threshold, 
#                                 cov_api=cov_api, 
#                                 chainName=chainName, 
#                                 network=network,
#                                 etherscan_api_key=etherscan_api_key
#                                 )

# early_pepe_whales.print_whales() 

# early_pepe_whales.threshold=200000 #change the threshold

# early_pepe_whales.print_whales() #whales different now.

# #`early_pepe_whales` has attributes like `whale_list`, i.e. can access via: early_pepe_whales.whale_list

# for _holder in early_pepe_whales.whale_list:
#     test_eq(_holder.get('is_whale'),True)


In [ ]:
# ###Saving and serialization

# #turn it into a dictionary
# early_pepe_whales_dict = earlywhales_to_dict(early_pepe_whales)
# print_whales_util(early_pepe_whales_dict)
# for _holder in early_pepe_whales.whale_list: test_eq(_holder.get('is_whale'),True)

# #Change the threshold
# threshold=500000
# early_pepe_whales_dict=earlywhales_dict_change_threshold(early_pepe_whales_dict, threshold)
# print_whales_util(early_pepe_whales_dict)
# for _holder in early_pepe_whales.whale_list: test_eq(_holder.get('is_whale'),True)

# #Save to json
# save_dict_to_json(early_pepe_whales_dict, 'early_pepe_whales_dict.json')

# #Load from json
# early_pepe_whales_dict = load_dict_from_json('early_pepe_whales_dict.json')

# print_whales_util(early_pepe_whales_dict) #should match the last print_whales_util call

#TODO: 

- Write more tests. Make the tests time efficient. (Similar to above but faster).
- 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()